### Verify the services are running


https://irsa-svc.domino-field is the service that connects to AWS to modify trust policies

In [ ]:
#First let us verify that the IRSA backend service is running
!curl -k https://irsa-svc.domino-field/healthz

A workspace connects to the https://irsa-svc.domino-field/ via a side-car container http://localhost:6003 . This side-car in injected via a domsed mutation. The same mutation also injects the appropriate environment variables in the `run` container to configure the ability to assume the irsa roles

In [ ]:
!curl http://localhost:6003/healthz

Verify that the web identity token with which the workspace authenticates with AWS is configured. This variable should be pre-populated with a JWT token

In [ ]:
#Fetch my mappings (Any user can do this)
import requests
import os
access_token_endpoint='http://localhost:8899/access-token'
resp = requests.get(access_token_endpoint)


token = resp.text
headers = {
             "Content-Type": "application/json",
             "Authorization": "Bearer " + token,
        }
endpoint='https://irsa-svc.domino-field/my_role_mappings'
resp = requests.get(endpoint,headers=headers,verify=False)
print(resp.text)


In [ ]:
!cat $AWS_WEB_IDENTITY_TOKEN_FILE

Now go to www.jwt.io and paste the above token. Note the `iss`. This is the token that boto3 api uses to implicitly authenticate with IAM. This occurs implicitly when the `AWS_WEB_IDENTITY_TOKEN_FILE` environment variable set.

Note the `iss`. This is the token that boto3 api uses to implicitly authenticate with IAM. This occurs implicitly when the `AWS_WEB_IDENTITY_TOKEN_FILE` environment variable set.

Note the `sub` is `system:serviceaccount:domino-compute:run-$DOMINO_RUN_ID` (your workspace service account is same as your run-id)
[Ex. `system:serviceaccount:domino-compute:run-646d27b817c28b5ea4b5b0c1`] 

`system:serviceaccount:domino-compute:run-646d27b817c28b5ea4b5b0c1` (your workspace service account) . This will be important later.

```json
{
  "aud": [
    "sts.amazonaws.com"
  ],
  "exp": 1685030736,
  "iat": 1684944336,
  "iss": "https://oidc.eks.us-west-2.amazonaws.com/id/xxxx",
  "kubernetes.io": {
    "namespace": "domino-compute",
    "pod": {
      "name": "run-646d27b817c28b5ea4b5b0c1-fxc2q",
      "uid": "63e373b0-4d62-4860-a329-baec1afcd3f8"
    },
    "serviceaccount": {
      "name": "run-646d27b817c28b5ea4b5b0c1",
      "uid": "ed97cfcf-58c6-4c1e-8929-71ff9bc335aa"
    }
  },
  "nbf": 1684944336,
  "sub": "system:serviceaccount:domino-compute:run-646d27b817c28b5ea4b5b0c1"
}
```

### AWS Config File

The `boto3` library looks for the config file in the following location `$AWS_CONFIG_FILE` which in your workspace
is set to `/var/run/.aws/config`

In [ ]:
!cat $AWS_CONFIG_FILE

This is file is created by the side-car container during startup. You can always refresh it using the command 
```shell
curl http://localhost:6003/refresh
```


For each profile in the asset account there is a corresponding role in the eks account

| AWS Profile (Asset Account)     | SRC AWS Profile (EKS Account) |
| ----------- | ----------- |
| `acme-list-bucket-role`      | `src_acme-list-bucket-role`       |
| `acme-read-bucket-role`  | `src_acme-read-bucket-role`       |
| `acme-update-bucket-role`   | `src_acme-read-bucket-role`       |

An example `src` profile looks like this:

```shell
[profile src_acme-list-bucket-role]
web_identity_token_file = /var/run/secrets/eks.amazonaws.com/serviceaccount/token
role_arn=arn:aws:iam::<EKS_ACCOUNT_NO>:role/acme-list-bucket-role
```
The corresponding asset profile looks like this:

```shell
[profile acme-list-bucket-role]
source_profile = src_acme-list-bucket-role
role_arn=arn:aws:iam::<ASSETS_ACCOUNT_NO>:role/acme-list-bucket-role
```

When the user assumes the asset profile `acme-list-bucket-role` it knows its `source_profile` is `src_acme-list-bucket-role`

The `src_acme-list-bucket-role` uses the `web_identity_token_file` to authenticate with AWS IAM which already has 
a trust relationship established with the OIDC provider who issues the web identity token. It uses this to assume the role
`role_arn` in the source profile `aws:iam::<EKS_ACCOUNT_NO>:role/acme-list-bucket-role`

The `aws:iam::<EKS_ACCOUNT_NO>:role/acme-list-bucket-role` in turn tries to assume the role `arn:aws:iam::<ASSETS_ACCOUNT_NO>:role/acme-list-bucket-role`
which is listed as the `role_arn` in the `[profile acme-list-bucket-role]`

### Bringing it all together

When the workspace starts up, the side-car invokes the backend IRSA service using the following code-snippet

In [ ]:
#Emulate Side-Car
import requests
import os
access_token_endpoint='http://localhost:8899/access-token'
resp = requests.get(access_token_endpoint)

os.environ['SSL_CERT_DIR']='/etc/ssl/certs/irsa'
token = resp.text
headers = {
             "Content-Type": "application/json",
             "Authorization": "Bearer " + token,
        }
endpoint='https://irsa-svc.domino-field/map_iam_roles_to_pod_sa'
print(f"Domino Run Id{os.environ['DOMINO_RUN_ID']}")
data = {"run_id": os.environ['DOMINO_RUN_ID']} ## It fetches this fom the downward api
resp = requests.post(endpoint,headers=headers,json=data,verify=os.environ['DOMINO_IRSA_SSL_CERT'])
resp.text

The backend IRSA service updates the trust policy file attached to each of the source roles being added to the config file.
The trust policy will look like this:
```json
{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Principal": {
                "Federated": "arn:aws:iam::<EKS_ACCOUNT>:oidc-provider/oidc.eks.us-west-2.amazonaws.com/id/<OIDC_PROVIDER_ID>"
            },
            "Action": "sts:AssumeRoleWithWebIdentity",
            "Condition": {
                "StringLike": {
                    "oidc.eks.us-west-2.amazonaws.com/id/<OIDC_PROVIDER_ID>:aud": "sts.amazonaws.com",
                    "oidc.eks.us-west-2.amazonaws.com/id/<OIDC_PROVIDER_ID>:sub": [
                        "*:domino-compute:run-646d27b817c28b5ea4b5b0c1",
                        "*:domino-compute:run-646f604955daf764547c5b77"
                    ]
                }
            }
        }
    ]
}
```

Note the `aud` and `sub` fields. This is the reason why the `src_acme-list-bucket-role` profile can assume the role
`arn:aws:iam::<EKS_ACCOUNT_NO>:role/acme-list-bucket-role`. If you remove the entries for your workspace from the trust policy,
the web identity token can authenticate with the AWS IAM but IAM will not allow it to assume the role.

Last each of the roles in the EKS Account have the following policies attached to them:

| Role    | Policies  |
| ----------- | ----------- |
| `acme-list-bucket-role`      | `acme-list-bucket-policy`       |
| `acme-read-bucket-role`  | `acme-read-bucket-policy`       |
| `acme-update-bucket-role`   | `acme-read-bucket-policy`       |

`acme-list-bucket-policy` looks like this:
```json
{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": "sts:AssumeRole",
            "Resource": "arn:aws:iam::<ASSETS_ACCOUNT_NO>:role/acme-list-bucket-role"
        }
    ]
}
```
The other two policies are similar


To summarize two things make it possible for the workspace to assume the role `arn:aws:iam::<EKS_ACCOUNT_NO>:role/acme-list-bucket-role`
(and the other two roles):

1. The web identity token which was issued by the oidc provider (`sub` provides the identity)
2. The `Condition` section of the trust policy attached to the roles which explictly declares which `sub` are allowed to 
assume the role.
3. The permission policies attached to the roles in the EKS account will be allowed to assume roles in the Assets account.

One last thing, the corresponding roles in the Assets Account ex. `arn:aws:iam::<ASSETS_ACCOUNT_NO>:role/acme-list-bucket-role`
need to have their trust policy to allow principals in the EKS account to assume it. This is the trust policy attached to
each of the roles `acme-list-bucket-role` , `acme-read-bucket-role` and `acme-update-bucket-role` 

```
{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Principal": {
                "AWS": "arn:aws:iam::<EKS_ACCOUNT_NO>:root"
            },
            "Action": "sts:AssumeRole"
        }
    ]
}
```

## Refreshing the .aws/config file

If the mappings have changed during a live workspace, you can refresh them using the call below. There is no need to restart the workspace

In [ ]:
!curl http://localhost:6003/refresh